In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
import joblib # IMPORTANTE

print("--- INICIO DEL PREPROCESAMIENTO ---")

# --- 1. Comprensión Inicial de los Datos ---
print("\n--- 1. Carga y Exploración Inicial ---")
data_dir_for_short = './data'
data_short_path = os.path.join(data_dir_for_short, 'data_short.csv')
full_data_path_original = './data/airline_passenger_satisfaction.csv' # Asumiendo que data también está en la raíz

try:
    if not os.path.exists(data_dir_for_short):
        os.makedirs(data_dir_for_short)
        print(f"Directorio '{data_dir_for_short}' creado.")

    if not os.path.exists(data_short_path):
        print(f"'{data_short_path}' no encontrado.")
        if os.path.exists(full_data_path_original):
            print(f"Creando '{data_short_path}' como una muestra de '{full_data_path_original}'.")
            df_full = pd.read_csv(full_data_path_original)
            if 'Unnamed: 0' in df_full.columns:
                 df_full = df_full.drop(columns=['Unnamed: 0'])
            df_full.sample(n=1000, random_state=42).to_csv(data_short_path, index=False)
            print(f"'{data_short_path}' creado con 1000 filas y sin índice de pandas en el archivo.")
            df = pd.read_csv(data_short_path)
        else:
            print(f"Error: Ni '{data_short_path}' ni '{full_data_path_original}' (para muestra) encontrados.")
            exit()
    else:
        df = pd.read_csv(data_short_path)
        if df.columns[0] == 'Unnamed: 0' and df.index.name is None : # Si la primera es un índice sin nombre
             df = pd.read_csv(data_short_path, index_col=0)


except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")
    exit()

print("Primeras 5 filas del dataset:")
print(df.head())
df.info()

# --- 2. Limpieza de Datos ---
print("\n--- 2. Limpieza de Datos ---")
print("\nValores faltantes por columna ANTES del tratamiento:")
print(df.isnull().sum())

if 'Arrival Delay in Minutes' in df.columns and df['Arrival Delay in Minutes'].isnull().any():
    median_arrival_delay = df['Arrival Delay in Minutes'].median()
    df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(median_arrival_delay)
    df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].astype(int)
    print(f"\n'Arrival Delay in Minutes' imputado con mediana ({median_arrival_delay}) y convertido a int.")
else:
    print("\nNo hay valores faltantes en 'Arrival Delay in Minutes' o la columna no existe.")

print("\nValores faltantes por columna DESPUÉS de la imputación:")
print(df.isnull().sum())

columns_to_drop = []
if 'id' in df.columns:
    columns_to_drop.append('id')
if 'ID' in df.columns:
    columns_to_drop.append('ID')

if columns_to_drop:
    df_cleaned = df.drop(columns=columns_to_drop, axis=1)
    print(f"\nColumnas {columns_to_drop} eliminadas.")
else:
    df_cleaned = df.copy()
    print("\nNo se encontraron columnas 'id' o 'ID' para eliminar.")

# --- 3. Transformación de Datos (Feature Engineering & Encoding) ---
print("\n--- 3. Transformación de Datos ---")
df_transformed = df_cleaned.copy()

if 'satisfaction' in df_transformed.columns and df_transformed['satisfaction'].dtype == 'object':
    satisfaction_map = {'neutral or dissatisfied': 0, 'satisfied': 1}
    df_transformed['satisfaction'] = df_transformed['satisfaction'].map(satisfaction_map)
    print("\nTarget 'satisfaction' mapeado a numérico.")

if 'Gender' in df_transformed.columns and df_transformed['Gender'].dtype == 'object':
    gender_map = {'Male': 0, 'Female': 1}
    df_transformed['Gender'] = df_transformed['Gender'].map(gender_map)
    print("Columna 'Gender' mapeada a numérico.")

if 'Customer Type' in df_transformed.columns and df_transformed['Customer Type'].dtype == 'object':
    customer_type_map = {'Loyal Customer': 1, 'disloyal Customer': 0}
    df_transformed['Customer Type'] = df_transformed['Customer Type'].map(customer_type_map)
    print("Columna 'Customer Type' mapeada a numérico.")

if 'Type of Travel' in df_transformed.columns and df_transformed['Type of Travel'].dtype == 'object':
    travel_type_map = {'Personal Travel': 0, 'Business travel': 1}
    df_transformed['Type of Travel'] = df_transformed['Type of Travel'].map(travel_type_map)
    print("Columna 'Type of Travel' mapeada a numérico.")

if 'Class' in df_transformed.columns and df_transformed['Class'].dtype == 'object':
    class_map = {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
    df_transformed['Class'] = df_transformed['Class'].map(class_map)
    print("Columna 'Class' mapeada a numérico (ordinal).")

print("\nVerificación de tipos después de mapeos:")
df_transformed.info()

if df_transformed.isnull().sum().any():
    print("\n¡Advertencia! Se encontraron NaNs después de los mapeos. Verificando...")
    print(df_transformed.isnull().sum()[df_transformed.isnull().sum() > 0])
    for col in ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']:
        if col in df_transformed.columns and df_transformed[col].isnull().any():
            print(f"Llenando NaNs en '{col}' con 0 (asumiendo fallo de mapeo o valor no esperado).")
            df_transformed[col] = df_transformed[col].fillna(0)

if 'Departure Delay in Minutes' in df_transformed.columns and 'Arrival Delay in Minutes' in df_transformed.columns:
    df_transformed['Total Delay in Minutes'] = df_transformed['Departure Delay in Minutes'] + df_transformed['Arrival Delay in Minutes']
    print("\nNueva característica 'Total Delay in Minutes' creada.")

numeric_cols_for_scaling = df_transformed.select_dtypes(include=np.number).columns.tolist()
cols_to_exclude_from_scaling = ['satisfaction']
cols_to_scale = [col for col in numeric_cols_for_scaling if col not in cols_to_exclude_from_scaling and col in df_transformed.columns]

# Directorio para guardar modelos de la app Flask
# ASUMIENDO QUE EL NOTEBOOK ESTÁ EN LA RAÍZ DEL PROYECTO, JUNTO A LA CARPETA 'app'
ML_MODELS_APP_DIR = 'app/ml_models' # <--- ¡¡¡CAMBIO IMPORTANTE SI EL NOTEBOOK ESTÁ EN LA RAÍZ!!!
                                    # Si el notebook está en 'notebooks/', usarías '../app/ml_models'

if not os.path.exists(ML_MODELS_APP_DIR):
    os.makedirs(ML_MODELS_APP_DIR)
    print(f"Directorio '{ML_MODELS_APP_DIR}' creado para los artefactos del modelo.")

scaler_object = None
if cols_to_scale:
    scaler_object = StandardScaler()
    df_transformed[cols_to_scale] = scaler_object.fit_transform(df_transformed[cols_to_scale])
    print(f"\nColumnas escaladas: {cols_to_scale}")

    scaler_path = os.path.join(ML_MODELS_APP_DIR, 'standard_scaler.pkl')
    cols_to_scale_path = os.path.join(ML_MODELS_APP_DIR, 'cols_to_scale.joblib')

    joblib.dump(scaler_object, scaler_path)
    print(f"Scaler guardado en: {scaler_path}")
    joblib.dump(cols_to_scale, cols_to_scale_path)
    print(f"Lista de columnas a escalar guardada en: {cols_to_scale_path}")
else:
    print("\nNo se encontraron columnas para escalar.")
    cols_to_scale_path = os.path.join(ML_MODELS_APP_DIR, 'cols_to_scale.joblib')
    joblib.dump([], cols_to_scale_path)
    print(f"Lista de columnas a escalar (vacía) guardada en: {cols_to_scale_path}")
    scaler_path = os.path.join(ML_MODELS_APP_DIR, 'standard_scaler.pkl')
    joblib.dump(None, scaler_path)
    print(f"Scaler (None) guardado en: {scaler_path}")

local_preprocessed_path = os.path.join(data_dir_for_short, 'data_short_preprocessed.csv')
try:
    df_transformed.to_csv(local_preprocessed_path, index=True)
    print(f"\nDataset preprocesado guardado localmente como '{local_preprocessed_path}'")
except Exception as e:
    print(f"\nError al guardar el archivo preprocesado localmente: {e}")

print("\n--- FIN DEL PREPROCESAMIENTO ---")
print("El DataFrame 'df_transformed' está listo para los siguientes pasos.")

--- INICIO DEL PREPROCESAMIENTO ---

--- 1. Carga y Exploración Inicial ---
Primeras 5 filas del dataset:
       id  Gender      Customer Type  Age   Type of Travel     Class  \
0   70172    Male     Loyal Customer   13  Personal Travel  Eco Plus   
1    5047    Male  disloyal Customer   25  Business travel  Business   
2  110028  Female     Loyal Customer   26  Business travel  Business   
3   24026  Female     Loyal Customer   25  Business travel  Business   
4  119299    Male     Loyal Customer   61  Business travel  Business   

   Flight Distance  Inflight wifi service  Departure/Arrival time convenient  \
0              460                      3                                  4   
1              235                      3                                  2   
2             1142                      2                                  2   
3              562                      2                                  5   
4              214                      3                    

In [34]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import joblib
import os

# --- 1. Cargar Datos Preprocesados ---
print("--- 1. Cargando datos preprocesados ---")
data_dir_for_short = './data'
preprocessed_file_path = os.path.join(data_dir_for_short, 'data_short_preprocessed.csv')

try:
    df_processed = pd.read_csv(preprocessed_file_path, index_col=0)
    print(f"Datos cargados correctamente desde '{preprocessed_file_path}'.")
except FileNotFoundError:
    print(f"Error: '{preprocessed_file_path}' no encontrado.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    exit()

# --- 2. Preparar Datos para Modelado (X e y) ---
print("\n--- 2. Preparando datos para modelado ---")
try:
    if 'satisfaction' not in df_processed.columns:
        raise KeyError("La columna 'satisfaction' no se encuentra. Revisa el CSV preprocesado.")
    X = df_processed.drop('satisfaction', axis=1)
    y = df_processed['satisfaction']
    print(f"Forma de X (características): {X.shape}")
    print(f"Forma de y (target): {y.shape}")

    # --- GUARDAR ORDEN DE CARACTERÍSTICAS ---
    # ASUMIENDO QUE EL NOTEBOOK ESTÁ EN LA RAÍZ DEL PROYECTO, JUNTO A LA CARPETA 'app'
    ML_MODELS_APP_DIR = 'app/ml_models' # <--- ¡¡¡CAMBIO IMPORTANTE SI EL NOTEBOOK ESTÁ EN LA RAÍZ!!!
                                        # Si el notebook está en 'notebooks/', usarías '../app/ml_models'

    if not os.path.exists(ML_MODELS_APP_DIR):
        os.makedirs(ML_MODELS_APP_DIR)
        print(f"Directorio '{ML_MODELS_APP_DIR}' creado para los artefactos del modelo.")

    feature_order_path = os.path.join(ML_MODELS_APP_DIR, 'feature_order.joblib')
    feature_order_list = list(X.columns)
    joblib.dump(feature_order_list, feature_order_path)
    print(f"Orden de características ({len(feature_order_list)} features) guardado en: {feature_order_path}")

except KeyError as ke:
    print(f"Error: {ke}")
    exit()
except Exception as e:
    print(f"Error al separar X e y: {e}")
    exit()

# --- 3. Dividir Datos en Entrenamiento y Prueba ---
print("\n--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)
print(f"Tamaño Entrenamiento: X={X_train.shape}, y={y_train.shape}")
print(f"Tamaño Prueba:        X={X_test.shape}, y={y_test.shape}")
print("\nDistribución de 'satisfaction' en Entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de 'satisfaction' en Prueba:")
print(y_test.value_counts(normalize=True))

--- 1. Cargando datos preprocesados ---
Datos cargados correctamente desde './data\data_short_preprocessed.csv'.

--- 2. Preparando datos para modelado ---
Forma de X (características): (1000, 23)
Forma de y (target): (1000,)
Orden de características (23 features) guardado en: app/ml_models\feature_order.joblib

--- 3. Dividiendo datos en conjuntos de Entrenamiento y Prueba ---
Tamaño Entrenamiento: X=(800, 23), y=(800,)
Tamaño Prueba:        X=(200, 23), y=(200,)

Distribución de 'satisfaction' en Entrenamiento:
satisfaction
0    0.56
1    0.44
Name: proportion, dtype: float64

Distribución de 'satisfaction' en Prueba:
satisfaction
0    0.56
1    0.44
Name: proportion, dtype: float64


In [35]:
# CELDA 3: COMPARACIÓN INICIAL DE MODELOS (OPCIONAL)

# Modelos de Clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
# from xgboost import XGBClassifier # Descomentar si tienes XGBoost
# from lightgbm import LGBMClassifier # Descomentar si tienes LightGBM

# Utilidades de Scikit-learn
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score # roc_auc_score ya debería estar
from time import time # Asegurarse que time esté importado aquí también

print("\n--- COMPARACIÓN INICIAL DE MODELOS (OPCIONAL) ---")
# Lista de modelos a evaluar
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "SVC (RBF)": SVC(kernel='rbf', probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest (Default)": RandomForestClassifier(random_state=42),
    "Gradient Boosting (Default)": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Gaussian Naive Bayes": GaussianNB(),
}

n_folds = 5
# X e y deben estar definidos desde la Celda 2
cv_strategy = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
scoring_metrics = ['accuracy', 'f1_weighted', 'roc_auc']
results_cv = {}

print(f"\n--- Evaluando {len(models)} modelos usando {n_folds}-Fold Cross-Validation (sobre datos COMPLETOS X, y) ---")
for model_name, model in models.items():
    start_time_cv = time() # Renombrar variable para evitar colisión si 'start_time' se usa después
    print(f"Evaluando: {model_name}...")
    try:
        # Usar X e y globales (todo el dataset preprocesado antes del split train/test para esta comparación general)
        cv_results_data = cross_validate(model, X, y, cv=cv_strategy, scoring=scoring_metrics, n_jobs=-1)
        results_cv[model_name] = {
            'Fit Time (mean)': cv_results_data['fit_time'].mean(),
            'Accuracy (mean)': cv_results_data['test_accuracy'].mean(),
            'F1 Weighted (mean)': cv_results_data['test_f1_weighted'].mean(),
            'AUC (mean)': cv_results_data['test_roc_auc'].mean(),
        }
        elapsed_time_cv = time() - start_time_cv
        print(f"  Completado en {elapsed_time_cv:.2f} segundos. AUC: {results_cv[model_name]['AUC (mean)']:.4f}")
    except Exception as e:
        print(f"  Error evaluando {model_name}: {e}")
        results_cv[model_name] = {metric: np.nan for metric in scoring_metrics + ['fit_time']}

results_cv_df = pd.DataFrame(results_cv).T.sort_values(by='AUC (mean)', ascending=False)
print("\nRanking de Modelos (Validación Cruzada General):")
print(results_cv_df[['AUC (mean)', 'F1 Weighted (mean)', 'Accuracy (mean)', 'Fit Time (mean)']].round(4))


--- COMPARACIÓN INICIAL DE MODELOS (OPCIONAL) ---

--- Evaluando 8 modelos usando 5-Fold Cross-Validation (sobre datos COMPLETOS X, y) ---
Evaluando: Logistic Regression...
  Completado en 0.18 segundos. AUC: 0.9278
Evaluando: KNN (k=5)...
  Completado en 0.34 segundos. AUC: 0.9368
Evaluando: SVC (RBF)...
  Completado en 0.88 segundos. AUC: 0.9579
Evaluando: Decision Tree...
  Completado en 0.76 segundos. AUC: 0.8903
Evaluando: Random Forest (Default)...
  Completado en 0.88 segundos. AUC: 0.9685
Evaluando: Gradient Boosting (Default)...
  Completado en 0.92 segundos. AUC: 0.9747
Evaluando: AdaBoost...
  Completado en 0.83 segundos. AUC: 0.9611
Evaluando: Gaussian Naive Bayes...
  Completado en 0.70 segundos. AUC: 0.9257

Ranking de Modelos (Validación Cruzada General):
                             AUC (mean)  F1 Weighted (mean)  Accuracy (mean)  \
Gradient Boosting (Default)      0.9747              0.9257            0.926   
Random Forest (Default)          0.9685              0.921

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score
import joblib
from time import time
import os

print("\n--- 4. AJUSTE DE HIPERPARÁMETROS PARA RANDOM FOREST (Enfoque Compañeros) ---")

base_rf_model = RandomForestClassifier(random_state=42)
param_distributions_rf_v3 = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10, 12],
    'min_samples_split': [15, 20, 25, 30],
    'min_samples_leaf': [8, 10, 12, 15],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}
n_folds_rs = 3
cv_strategy_rs = StratifiedKFold(n_splits=n_folds_rs, shuffle=True, random_state=42)
n_iterations_search = 75

random_search_rf = RandomizedSearchCV(
    estimator=base_rf_model,
    param_distributions=param_distributions_rf_v3,
    n_iter=n_iterations_search,
    scoring='f1_weighted',
    cv=cv_strategy_rs,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print(f"\nIniciando RandomizedSearchCV para RandomForestClassifier...")
print(f"Probando {random_search_rf.n_iter} combinaciones de parámetros con {n_folds_rs}-fold CV.")
start_time = time()
random_search_rf.fit(X_train, y_train)
end_time = time()
print(f"\nRandomizedSearchCV completado en {(end_time - start_time):.2f} segundos.")

print("\n--- 5. Resultados del Ajuste de Hiperparámetros (RandomForest) ---")
print(f"Mejor puntuación F1-Weighted (media en CV sobre datos de entrenamiento): {random_search_rf.best_score_:.4f}")
print("Mejores hiperparámetros encontrados:")
print(random_search_rf.best_params_)

best_rf_model = random_search_rf.best_estimator_

if hasattr(best_rf_model, 'feature_importances_'):
    feature_importances_rf = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    print("\nImportancia de las Características (Random Forest Optimizado):")
    print(feature_importances_rf.head(10))
else:
    print("\nEl modelo no tiene el atributo 'feature_importances_'.")

print("\n--- 6. Evaluación Final del Mejor Modelo RandomForest en el CONJUNTO DE PRUEBA ---")
y_pred_test_rf = best_rf_model.predict(X_test)
y_proba_test_rf = best_rf_model.predict_proba(X_test)[:, 1]
accuracy_test_rf = accuracy_score(y_test, y_pred_test_rf)
f1_test_rf = f1_score(y_test, y_pred_test_rf, average='weighted')
auc_test_rf = roc_auc_score(y_test, y_proba_test_rf)

print("Métricas en el conjunto de prueba (Random Forest Optimizado):")
print(f"Accuracy: {accuracy_test_rf:.4f}")
print(f"F1 Score (Weighted): {f1_test_rf:.4f}")
print(f"AUC: {auc_test_rf:.4f}")
print("\nReporte de Clasificación detallado (Random Forest Optimizado en Test):")
print(classification_report(y_test, y_pred_test_rf))

# --- 7. GUARDAR EL MODELO ENTRENADO PARA LA APP FLASK ---
print("\n--- 7. Guardando el modelo RandomForest optimizado para la aplicación ---")

# ASUMIENDO QUE EL NOTEBOOK ESTÁ EN LA RAÍZ DEL PROYECTO, JUNTO A LA CARPETA 'app'
ML_MODELS_APP_DIR = 'app/ml_models' # <--- ¡¡¡CAMBIO IMPORTANTE SI EL NOTEBOOK ESTÁ EN LA RAÍZ!!!
                                    # Si el notebook está en 'notebooks/', usarías '../app/ml_models'
                                    
if not os.path.exists(ML_MODELS_APP_DIR):
    os.makedirs(ML_MODELS_APP_DIR)
    print(f"Directorio '{ML_MODELS_APP_DIR}' creado para los artefactos del modelo.")

MODEL_APP_FILENAME = 'random_forest_satisfaction_model.pkl' # Nombre del modelo para la app

final_model_path_for_app = os.path.join(ML_MODELS_APP_DIR, MODEL_APP_FILENAME)

try:
    joblib.dump(best_rf_model, final_model_path_for_app)
    print(f"Modelo guardado exitosamente para la app como '{final_model_path_for_app}'")
except Exception as e:
    print(f"Error al guardar el modelo para la app: {e}")

# --- 8. Calcular Overfitting (No cambia) ---
# ... (el resto del código de la Celda 4 para el overfitting se mantiene igual) ...
print("\n--- 8. Calculando Overfitting para RandomForest Optimizado ---")
y_pred_train_rf = best_rf_model.predict(X_train)
y_proba_train_rf = best_rf_model.predict_proba(X_train)[:, 1]
train_auc_rf = roc_auc_score(y_train, y_proba_train_rf)
train_f1_rf = f1_score(y_train, y_pred_train_rf, average='weighted')

print(f"AUC en Conjunto de Entrenamiento: {train_auc_rf:.4f}")
print(f"F1 (weighted) en Conjunto de Entrenamiento: {train_f1_rf:.4f}")
print(f"AUC en Conjunto de Prueba:        {auc_test_rf:.4f}")
print(f"F1 (weighted) en Conjunto de Prueba:        {f1_test_rf:.4f}")

overfitting_abs_f1 = train_f1_rf - f1_test_rf
overfitting_abs_auc = train_auc_rf - auc_test_rf
print(f"\nDiferencia Absoluta F1 (Train - Test): {overfitting_abs_f1:.4f}")
print(f"Diferencia Absoluta AUC (Train - Test): {overfitting_abs_auc:.4f}")

if overfitting_abs_f1 > 0.05 :
    print("\nAdvertencia: El modelo podría estar sobreajustando según el F1-score (diferencia > 5%). ❌")
else:
    print("\nEl modelo parece tener un buen balance entre entrenamiento y prueba según el F1-score. ✅")
if overfitting_abs_auc > 0.05 :
    print("Advertencia: El modelo podría estar sobreajustando según el AUC (diferencia > 5%). ❌")
else:
    print("El modelo parece tener un buen balance entre entrenamiento y prueba según el AUC. ✅")

print("\n--- PROCESO COMPLETADO CON RANDOMFOREST + RANDOMIZEDSEARCHCV ---")


--- 4. AJUSTE DE HIPERPARÁMETROS PARA RANDOM FOREST (Enfoque Compañeros) ---

Iniciando RandomizedSearchCV para RandomForestClassifier...
Probando 75 combinaciones de parámetros con 3-fold CV.
Fitting 3 folds for each of 75 candidates, totalling 225 fits

RandomizedSearchCV completado en 2.82 segundos.

--- 5. Resultados del Ajuste de Hiperparámetros (RandomForest) ---
Mejor puntuación F1-Weighted (media en CV sobre datos de entrenamiento): 0.8882
Mejores hiperparámetros encontrados:
{'n_estimators': 200, 'min_samples_split': 15, 'min_samples_leaf': 8, 'max_features': 'log2', 'max_depth': 7, 'bootstrap': True}

Importancia de las Características (Random Forest Optimizado):
                   feature  importance
11         Online boarding    0.186885
4                    Class    0.149277
3           Type of Travel    0.122864
13  Inflight entertainment    0.084234
6    Inflight wifi service    0.080817
12            Seat comfort    0.055730
5          Flight Distance    0.040375
14   